In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gc import collect as collect_garbage
from psutil import virtual_memory
from os import scandir

In [18]:
folder = "../01-rpeaks/"

In [48]:
# Clean up the RR intervals
for i in range(1,8):
    print("Starting now with infant " + str(i))

    # Get the names of the rpeak files
    files = sorted( [file.name for file in scandir(folder) if str(i) in file.name] )
    
    # Concatenate all of these dataframes
    df = pd.read_csv( folder + files[0] )
    for file in files[1:]:
        df = df.append( pd.read_csv(folder + file) )
        collect_garbage()
    
    # Calculate the RR intervals
    df["interval"] = df["time"].diff()

    # Recalculate RR intervals while ignoring beats whose RR intervals are < 0.25 seconds
    filt = df["interval"] >= 0.25

    buffer = pd.DataFrame.copy(df.loc[filt], deep=True)
    buffer.reset_index(inplace=True, drop=True)
    buffer["interval"] = buffer["time"].diff()

    del df; collect_garbage()
    df = pd.DataFrame.copy(buffer, deep=True)
    del buffer; collect_garbage()

    # Remove intervals of length greater than 5 seconds (arbitrary threshold)
    # The resulting gaps will just be treated as missing data
    filt = df["interval"] <= 5
    df = df.loc[filt]
    df.reset_index(inplace=True, drop=True)

    # Remove any remaining multiple intervals
    

    break

Starting now with infant 1


In [47]:
buffer

1         3.456006e+05
2         3.456010e+05
3         3.456014e+05
4         3.456018e+05
5         3.456022e+05
              ...     
710152    1.641598e+06
710153    1.641599e+06
710154    1.641599e+06
710155    1.641599e+06
710156    1.641600e+06
Name: time, Length: 3439809, dtype: float64

In [38]:
# This cell tests if concatenating the dataframes works as expected
df.reset_index(drop=True,inplace=True)

print(df)

df1 = pd.read_csv(folder+files[0])
df2 = pd.read_csv(folder+files[1])
df3 = pd.read_csv(folder+files[2])
df4 = pd.read_csv(folder+files[3])
df5 = pd.read_csv(folder+files[4])

len(df1["time"]) + len(df2["time"]) + len(df3["time"]) + len(df4["time"]) + len(df5["time"])

In [ ]:
# This cell tests if recalculating RR intervals while ignoring beats less than
# 0.25 seconds was a success. If so, the result should be 0
sum(df["interval"] < 0.25)